# It's Tree Huggin' Time!

#### Before you begin:
* This notebook will establish a MySQL connection, create a database, its
associated tables, without the use of the MySQL UI. Please make sure to 
have mysqlclient installed by running 
```pip install mysqlclient``` 
in your python runtime environment.
* The data set for NYC Trees is rather large and is not stored in this repository, please go to [Kaggle](https://www.kaggle.com/new-york-city/ny-2015-street-tree-census-tree-data/version/12) and download to the resources folder within this repository.
* Remember to insert you password in mysql_scr.py (please do not alter the other variables)
* Let's get tree huggin'

In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import mysql_scr
import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import os

In [2]:
# set up mysql initial connections
init_string = (f"root:{mysql_scr.pw}@localhost")
pre_engine = create_engine(f"mysql://{init_string}")

In [3]:
# create mysql db
pre_engine.execute(f"{mysql_scr.create_db}")

In [4]:
# initialize new mysql db connection
engine = create_engine(f"mysql://{init_string}/tree_db")
engine.execute(f"{mysql_scr.create_tbl}")

In [5]:
# check if tables exist...
engine.table_names()

['nyc_tree', 'tree_species']

In [6]:
# set file
file = os.path.join("Resources","2015-street-tree-census-tree-data.csv")

In [7]:
tree_data = pd.read_csv(file)
tree_data.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27T00:00:00,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03T00:00:00,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05T00:00:00,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05T00:00:00,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30T00:00:00,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


In [8]:
#obtain desired columns
tree_data = tree_data[['postcode','spc_common','spc_latin','tree_dbh']]
#clean data
tree_data_drop = tree_data.dropna()
tree_data_clean = tree_data_drop.drop_duplicates()

tree_data_clean.head()

,postcode,spc_common,spc_latin,tree_dbh
0,11375,red maple,Acer rubrum,3
1,11357,pin oak,Quercus palustris,21
2,11211,honeylocust,Gleditsia triacanthos var. inermis,3
3,11211,honeylocust,Gleditsia triacanthos var. inermis,10
4,11215,American linden,Tilia americana,21


In [9]:
#obtain average tree diameter
tree_info = tree_data_clean.groupby(['postcode','spc_common'])
tree_diam = pd.DataFrame(tree_info['tree_dbh'].mean())

#obtain number of tree type in postcode

tree_quantity = pd.DataFrame(tree_data_clean.groupby(['postcode','spc_common']).size())

tree_summary = tree_quantity.join(tree_diam).reset_index()
tree_summary = tree_summary.rename(columns={0:'tree_count'})
tree_summary.head(2)

,postcode,spc_common,tree_count,tree_dbh
0,83,American beech,2,7.500000
1,83,American elm,44,23.522727


In [10]:
#rename columns to match database table
tree_transformed = tree_summary.rename(columns = {'postcode':'zip_code',
                              'spc_common': 'species_nm',
                              'tree_count': 'count_tree',
                              'tree_dbh': 'avg_diameter'})
tree_transformed.head()

,zip_code,species_nm,count_tree,avg_diameter
0,83,American beech,2,7.500000
1,83,American elm,44,23.522727
2,83,Callery pear,3,7.000000
3,83,Chinese elm,3,21.333333
4,83,Japanese zelkova,24,20.916667


In [11]:
#load dataframe to database
tree_transformed.to_sql(name='nyc_tree',con=engine,if_exists='append',index=False)

In [15]:
# validate import
result = engine.execute('Select * from nyc_tree limit 20')
for r in result:
    print(r)

(1, 83, 'American beech', 2, 8)
(2, 83, 'American elm', 44, 24)
(3, 83, 'Callery pear', 3, 7)
(4, 83, 'Chinese elm', 3, 21)
(5, 83, 'Japanese zelkova', 24, 21)
(6, 83, 'London planetree', 10, 11)
(7, 83, 'Sophora', 2, 11)
(8, 83, 'ginkgo', 27, 15)
(9, 83, 'honeylocust', 6, 17)
(10, 83, 'littleleaf linden', 3, 5)
(11, 83, 'northern red oak', 1, 7)
(12, 83, 'pin oak', 35, 21)
(13, 10001, "'Schubert' chokecherry", 3, 4)
(14, 10001, 'American elm', 14, 11)
(15, 10001, 'American hophornbeam', 1, 2)
(16, 10001, 'American linden', 5, 5)
(17, 10001, 'Amur maple', 1, 4)
(18, 10001, 'Callery pear', 18, 10)
(19, 10001, 'Chinese elm', 1, 14)
(20, 10001, 'English oak', 1, 4)
